This notebook translates technology subsidy in the scenarios to the required format in policy impact tables

v2 (12 May 2023) --> paste it in policy_step1_technologytax and then *ACCEPT* through TEAM UI

v3 (15 Aug 2023) --> added a year dimension so you can vary subsidies by year

In [168]:
import pandas as pd
import numpy as np

In [169]:
scenario_names = ['OMOKA', 'ELECTRICDREAMS', 'KUJINAO', 'STREETFIGHTING']

In [170]:
# you only need this for PurchasePrice and second-hand import flag
def return_technology():
    technology = pd.read_excel(f'./../TEAM-Kenya_data_out/original-team-uk-data/Technology.xlsx')
    return technology

In [171]:
def return_policy_impacts():
    policy_impacts = pd.read_excel(f'./../TEAM-Kenya_data_out/original-team-uk-data/Policy_Impacts.xlsx')
    return policy_impacts

In [172]:
def return_policy_step1_techtax():
    policy_step1 = pd.read_excel(f'./../TEAM-Kenya_data_out/original-team-uk-data/Policy_Step1_TechnologyTax.xlsx')
    return policy_step1

In [173]:
def return_base_rates_policy_impacts(policy_impacts, technology, base_rates = {'ImportDuty': 0.25, 'ExciseDuty': 0.2, 'VAT': 0.16}):

    for techid in technology.TechID.unique().tolist():

        set_inds = policy_impacts[policy_impacts.TechID == techid].index.tolist()

        policy_impacts.loc[set_inds, 'PurchaseTax'] = base_rates['ImportDuty']*technology[technology.TechID == techid]['PurchasePrice'].item() + base_rates['ExciseDuty']*(1+base_rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item() + base_rates['VAT']*(1+base_rates['ExciseDuty'])*(1+base_rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item()

    return policy_impacts

In [174]:
# subsidies can be by FuelID, by VehTypeID.
# Narrative: Omoka focussed on growth of public e-mobility so subsidies highest for e-buses. Electric dreams has subsidies highest for e-cars and E2Ws. Street fighting = no subsidies. Kujinao has some subsidies for second-hand imported electric buses

# purhcasetax dictionary, by scenario, by fuelid (12=elec), by vehtypeid (1=bike, 2=moto, 3=car, 4=bus, 5=truck, 6=train, 7=ship, 8=plane), by SHI status (SecondHandImportFlag=0 for non-SHI, = 1 for SHI) by variable:value key value pair

# omoka - the most subisidsed electric buses
purchasetax_dict = {'OMOKA': {
        12: {
            2: {
                0: {
                    'ImportDuty': {2012: 0.25, 2028: 0},
                    'ExciseDuty': {2012: 0.1, 2040: 0},
                    'VAT': {2012: 0.16}
                }
            },
            3: {
                0: {
                    'ImportDuty': {2012: 0.25, 2040: 0.15},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2040: 0.15},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.1
                }
            },
            4: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0.05, 2028: 0},
                    'VAT': {2012: 0.16, 2025: 0}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0.05, 2028: 0},
                    'VAT': {2012: 0.16, 2025: 0}
                }
            },
            5: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0.15, 2035: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0.05, 2028: 0},
                    'VAT': {2012: 0.16, 2025: 0}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0.15, 2035: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0.05, 2028: 0},
                    'VAT': {2012: 0.16, 2025: 0}
                }
            },
            6: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0},
                    'VAT': {2012: 0.16, 2023: 0}
                }
            }
        }
    },

            # electric dreams - the most subsidised EVs.               
            'ELECTRICDREAMS': {
        12: {
            2: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0},
                    'VAT': {2012: 0.16, 2023: 0}
                }
            },
            3: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0},
                    'VAT': {2012: 0.16, 2023: 0}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0},
                    'VAT': {2012: 0.16, 2023: 0}
                }
            },
            4: {
                0: {
                    'ImportDuty': {2012: 0.25, 2040: 0.15},
                    'ExciseDuty': {2012: 0.1, 2040: 0.05},
                    'VAT': {2012: 0.16, 2040: 0.1}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2040: 0.15},
                    'ExciseDuty': {2012: 0.1, 2040: 0.05},
                    'VAT': {2012: 0.16, 2040: 0.1}
                }
            },
            5: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0.15, 2035: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0.05, 2028: 0},
                    'VAT': {2012: 0.16, 2025: 0}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0.15, 2035: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0.05, 2028: 0},
                    'VAT': {2012: 0.16, 2025: 0}
                }
            },
            6: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0},
                    'ExciseDuty': {2012: 0.1, 2023: 0},
                    'VAT': {2012: 0.16, 2023: 0}
                }
            }
        }
    },
                         
            # kujinao - reflects current kenyan policy as of 2023
                    
            'KUJINAO': {
        12: {
            2: {
                0: {
                    'ImportDuty': {2012: 0.25, 2030: 0},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                }
            },
            3: {
                0: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                },
                1: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                }
            },
            4: {
                0: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                },
                1: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                }
            },
            5: {
                0: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                },
                1: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                }
            },
            6: {
                0: {
                    'ImportDuty': {2012: 0.25},
                    'ExciseDuty': {2012: 0.1},
                    'VAT': {2012: 0.16}
                }
            }
        }
    },
                                
            # streetfighting - low growth results in high taxes stifling growth in e-mob markets?
                    
            'STREETFIGHTING': {
        12: {
            2: {
                0: {
                    'ImportDuty': {2012: 0.25, 2025: 0.3},
                    'ExciseDuty': {2012: 0.15},
                    'VAT': {2012: 0.16}
                }
            },
            3: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                }
            },
            4: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                }
            },
            5: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                },
                1: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                }
            },
            6: {
                0: {
                    'ImportDuty': {2012: 0.25, 2023: 0.3},
                    'ExciseDuty': {2012: 0.1, 2020: 0.2},
                    'VAT': {2012: 0.16, 2028: 0.2}
                }
            }
        }
    }}

In [175]:
# function to return interpolated values in purchase tax dictionaries (above)
def return_interpolated_value(y, fuelID, vehicleID, secondhandflagID, scenario, tax_type, purchasetax_data):
    
    # return a dict of {year: value}
    PTdict = purchasetax_data[scenario][fuelID][vehicleID][secondhandflagID][tax_type]
    
    if y in PTdict:
        x = PTdict[y]
    else:
        # if it's not the last one in the sorted list
        if sorted(list(PTdict.keys())+[y]).index(y) < len(list(PTdict.keys())+[y]) - 1:
            x = PTdict[list(PTdict.keys())[sorted(list(PTdict.keys())+[y]).index(y)-1]] + (y - list(PTdict.keys())[sorted(list(PTdict.keys())+[y]).index(y)-1])/(list(PTdict.keys())[sorted(list(PTdict.keys())+[y]).index(y)] - list(PTdict.keys())[sorted(list(PTdict.keys())+[y]).index(y)-1]) * (PTdict[list(PTdict.keys())[sorted(list(PTdict.keys())+[y]).index(y)]] - PTdict[list(PTdict.keys())[sorted(list(PTdict.keys())+[y]).index(y)-1]])
        else:
            if sorted(list(PTdict.keys())+[y]).index(y) == 0:
                x = 0
            else:
                x = PTdict[list(PTdict.keys())[-1]]
    return np.round(x, 2)

In [176]:
# function to change those rates based on purchasetax_dictionary
def return_changed_policy_impacts(year, scenario, policy_impacts, technology, purchasetax_dict):

    for fuelid in list(purchasetax_dict[scenario].keys()):

        for vehicletypeid in list(purchasetax_dict[scenario][fuelid].keys()):

            for shi_status in list(purchasetax_dict[scenario][fuelid][vehicletypeid].keys()):

                # return dictionary of rates for this year
                rates = {}
                for rate in list(purchasetax_dict[scenario][fuelid][vehicletypeid][shi_status].keys()):
                    rates[rate] = return_interpolated_value(year, fuelid, vehicletypeid, shi_status, scenario, rate, purchasetax_dict)
                
                # return list of techIDs to which this set of rates applies
                techids = technology[(technology.FuelID == fuelid) & (technology.VehTypeID == vehicletypeid) & (technology.SecondHandImportFlag == shi_status)]['TechID'].tolist()
                
                for techid in techids:

                    set_inds = policy_impacts[(policy_impacts.Year == year) & (policy_impacts.TechID == techid)].index.tolist()

                    policy_impacts.loc[set_inds, 'PurchaseTax'] = rates['ImportDuty']*technology[technology.TechID == techid]['PurchasePrice'].item() + rates['ExciseDuty']*(1+rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item() + rates['VAT']*(1+rates['ExciseDuty'])*(1+rates['ImportDuty'])*technology[technology.TechID == techid]['PurchasePrice'].item()

    return policy_impacts

In [177]:
# re-write taxation policy with scenarios

for scenario_name in scenario_names:
    
    print(f'doing {scenario_name}')
    
    technology = return_technology()
    # policy_impacts = return_policy_impacts()
    policy_step1 = return_policy_step1_techtax()

    # write base rates
    policy_step1 = return_base_rates_policy_impacts(policy_step1, technology)

    # change for scenario for each year
    for year in range(2012, 2051):
        policy_step1 = return_changed_policy_impacts(year, scenario_name, policy_step1, technology, purchasetax_dict)

    # save
    policy_step1.to_excel(f'./../TEAM-Kenya_data_out/scenarios/Policy_Step1_TechnologyTax{scenario_name}.xlsx', index=False)

doing OMOKA
doing ELECTRICDREAMS
doing KUJINAO
doing STREETFIGHTING
